## API AlphaVantage - Intradiario

#### Importar las librerías necesarias

In [26]:
import requests
from decouple import config
import json
import pandas as pd
import psycopg2

#### Leer la contraseña y el token desde la variable de entorno

In [27]:
my_host = config('DATABASE_HOST')
port = config('DATABASE_PORT')
password = config('DATABASE_PASSWORD')
database_name = config('DATABASE_NAME')
user = config('DATABASE_USER')
token = config('ALPHAVANTAGE_TOKEN')

#### Definir la función para obtener datos de AlphaVantage

In [29]:
def getIntra(symbol, interval, size): 
    function = 'TIME_SERIES_INTRADAY'
    url = 'https://www.alphavantage.co/query'
    parametros = {
        'function': function, 
        'symbol': symbol, 
        'interval': interval, 
        'outputsize': size, 
        'datatype': json, 
        'apikey': token
    }
    
    r = requests.get(url, params=parametros)
    data = r.json() # Obtener el JSON
    series_key = f"Time Series ({interval})"
    if series_key in data:
        dataDF = pd.DataFrame.from_dict(data[series_key], orient='index') # Convertir a DataFrame
        dataDF = dataDF.astype(float)
        dataDF.index.name = 'Fecha'
        dataDF.columns = ['Apertura', 'Maximo', 'Minimo', 'Cierre', 'Volumen']
        dataDF = dataDF.sort_index(ascending=True)
        dataDF.index = pd.to_datetime(dataDF.index)
        return dataDF
    else:
        return pd.DataFrame()

acciones = ['AMZN', 'MELI', 'AAPL', 'GOOGL', 'MSFT'] # Lista de símbolos de acciones que se van a consultar


#### Crear e imprimir el Dataframe

In [30]:
combined_data = pd.DataFrame() # Inicializar un DataFrame vacío para combinar los datos

# Obtener y combinar los datos de las acciones en el DataFrame
for accion in acciones:
    dataDF = getIntra(symbol=accion, interval='5min', size='compact')
    if not dataDF.empty:
        dataDF['Simbolo'] = accion # Agregar una columna para el símbolo de la acción
        combined_data = pd.concat([combined_data, dataDF])

combined_data = combined_data.round(6) # Redondear los valores a seis decimales

print(combined_data) # Imprimir el DataFrame combinado


Empty DataFrame
Columns: []
Index: []


#### Conectarse con Amazon Redshift y creación de tabla (si no existe)

In [20]:

#Conectar a la base de datos
try:
    conn = psycopg2.connect(
        host=my_host, 
        port=port, 
        database=database_name, 
        user=user, 
        password=password
    )
    print("Conexión a la base de datos exitosa")

    # Crear tabla
    try:
        cur = conn.cursor()
        cur.execute("""
        CREATE TABLE IF NOT EXISTS hvzambrana_coderhouse.alphavantage (
            simbolo VARCHAR(10) DISTKEY NOT NULL,
            fecha TIMESTAMP SORTKEY NOT NULL,
            apertura FLOAT(15) NULL,
            maximo FLOAT(15) NULL,
            minimo FLOAT(15) NULL,
            cierre FLOAT(15) NULL,
            volumen FLOAT(15) NULL
        );
        """)
        conn.commit()
        print("Tabla creada exitosamente")
    except Exception as e:
            print("Error al crear la tabla")
            print(e)

except Exception as e:
    print("Error al crear la conexión a la base de datos")
    print(e)
finally:
    if conn is not None:
        conn.close()
        print("Conexión cerrada correctamente")

Conexión a la base de datos exitosa
Tabla creada exitosamente
Conexión cerrada correctamente
